In [1]:
#################################### IMPORTS ###################################################
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import tensorflow as tf
import datetime
from keras.backend.tensorflow_backend import set_session

Using TensorFlow backend.


In [2]:
##################### INICIALIZACAO DE VARIAVEIS ################################################
index_arquivo = ['preco', 'hr_int']#, 'preco_pon', 'qnt_soma', 'max', 'min', 'IND', 'ISP'] #index do arquivo
steps = [] # 9h04 -> 17h50 a cada 5 segundos 
epocas = 1000 #quantidade de vezes que vai rodar todos os dias
janela = 10 #janela de valores
n_variaveis = len(index_arquivo) #'preco', 'hr_int', 'preco_pon', 'qnt_soma', 'max', 'min', 'IND', 'ISP'
l = n_variaveis-1
tam_janela = n_variaveis*janela
n_entradas = n_variaveis * janela + 3 #ncont, valor, posicao e inputs
n_neuronios = 216 #numero de neuronios da camada escondida
lim_cont = 5
n_saidas = 4*lim_cont+1   #numero de saidas da rede (compra, vende, segura)
custo = 1.06/2 #custo da operao
melhor_reward = 0

versao_arquivo = 2

carregar_pesos = False
carregar_epoca_epsilon = False
epoca_init = 0
if carregar_epoca_epsilon:
    file = open("./epoca_epsilon.txt", "r")
    valores = file.read().split(',')
    epoca_init = int(valores[0])
    epsilon = float(valores[1])
    file.close()
else:
    epsilon = 1. #valor de epsilon
epsilon_min = 0.01 #valor minimo de epsilon
epsilon_decay = (epsilon - epsilon_min) / (epocas - epoca_init) #o valor que vai retirado do epsilon por epoca

rewards = [0] #variavel para guardar rewards
plotx = [0] #variavel para guardar valores a serem plotados do eixo x

In [3]:
####################### LEITURA DOS DADOS #######################################################
dias = 220

caminho_arquivo = ('./consolidado.csv')
arquivo = pd.read_csv(caminho_arquivo) #le arquivo
inputs = arquivo[index_arquivo]
print(inputs['preco'][0:2])
if versao_arquivo == 1: #se quiser usar apenas os dias com IND e ISP
    inputs = inputs[inputs['IND'] != 0]
    arquivo = arquivo[arquivo['IND'] != 0]

dt = arquivo['dt'].values #cria coluna apenas dos dias

steps = []
ultimo_dia = 0
dias_para_rodar = [] #variavel para colocar os dias a serem rodados
j = 0
hr = []
h = 0
n_steps = 106 #especifico pra 5min
for i in range( 0, len(dt) ):    
    if (dt[i] != ultimo_dia):
        steps.append(i) #numero de linhas entre dias
        ultimo_dia = dt[i]
        dias_para_rodar.append(j) #numero do dia
        j += 1
        h = 0
    hr.append(h)
    h += 1
step_max = np.amax(hr)
batch = step_max + 1

#normalizacao dos dados
hr = hr/step_max
inputs['hr_int'] = hr
pmean = np.mean( inputs.loc[:dias*batch, inputs.columns[0]] ) #define valor minimo do preco  #step = 106
pstd = np.std( inputs.loc[:dias*batch, inputs.columns[0]] ) #define valor maximo do preco
for i in range( inputs.shape[1] ): #roda normalizo para todas as colunas
    imean = np.mean( inputs.loc[:dias*batch, inputs.columns[i]] ) #pega valor maximo
    istd = np.std( inputs.loc[:dias*batch, inputs.columns[i]] ) #pega valor minimo
    
    inputs.loc[:, inputs.columns[i]] = ( inputs.loc[:, inputs.columns[i]] - imean ) / istd #normaliza prs
    
mini_batch_size = int(dias*batch*0.1)

D:\anaconda\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0    3907.5
1    3900.0
Name: preco, dtype: float64


D:\anaconda\lib\site-packages\pandas\core\indexing.py:635: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [ ]:
########################################   BIBLIOTECAS ####################################
import random
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
from keras import backend as K            #importa backend para clear_session()


##################### MODELO DQN ####################################################
class DQNAgent:
    ########################### INICIALIZA ###########################################
    def __init__(self, state_size, action_size, epsilon, janela, n_neuronios, n_variaveis):
        self.state_size = state_size
        self.n_neuronios = n_neuronios
        self.action_size = action_size
        self.janela = janela
        self.n_variaveis = n_variaveis
        self.limpa_memoria_dia()
        self.limpa_memoria_epoca()
        self.gamma = 0.97       # discount rate
        self.epsilon = epsilon  # exploration rate
        self.learning_rate = 1e-4
        self.model = self.cria_modelo()
        self.model.summary()
        self.state = []
        self.next_state = []
        self.janela_precos = []
        self.prox_janela_precos = []

################################# REDE NEURAL ###########################################
    def cria_modelo(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        """
        model.add(Dense(self.n_neuronios, input_dim=self.state_size, activation='relu')) #camada de entrada (escondida)
        model.add(Dense(self.n_neuronios, activation='relu')) #camada escondida
        model.add(Dense(self.n_neuronios, activation='relu')) #camada escondida
        model.add(Dense(self.action_size, activation='softmax')) #camada de saida
        model.compile(loss='categorical_crossentropy', optimizer=tf.train.AdamOptimizer(learning_rate=self.learning_rate)) #compilador
        """
        
        model.add(Dense(320, input_dim=self.state_size, activation='relu')) #camada de entrada (escondida)
        model.add(Dense(256, activation='relu')) #camada escondida
        model.add(Dense(256, activation='relu')) #camada escondida
        model.add(Dense(256, activation='relu')) #camada escondida
        model.add(Dense(self.action_size, activation='linear')) #camada de saida
        #model.compile(loss='mse', optimizer=tf.train.AdamOptimizer(learning_rate=self.learning_rate)) #compilador
        Adam = optimizers.Adam(lr=self.learning_rate)
        model.compile(loss='mean_squared_error', optimizer=Adam)
        return model

    def limpa_memoria_dia(self):
        self.janela_precos = np.zeros(n_variaveis*janela)
        self.prox_janela_precos = np.zeros(n_variaveis*janela)
        self.state = []
        self.next_state = []
    
    def limpa_memoria_epoca(self):
        self.memory = []
        
    def toma_acao(self):
        if np.random.rand() <= self.epsilon: #se o numero aleatorio for menor que o epsilon
            return random.randrange(self.action_size) #retorna acao aleatoria     
        act_values = self.model.predict(modelo.state) #calcula qual a melhor acao
        return np.argmax(act_values[0])  # returns action
    
    def treina_modelo(self, batch_size=mini_batch_size):
        minibatch = random.sample(self.memory, batch_size)
        
        x = np.zeros((batch_size+1, self.state_size))
        y = np.zeros((batch_size+1, self.action_size))
        i = 0
        for acao, reward, estado, prox_estado, done in minibatch:        
            
            target = reward
            if not done:
                target = (reward + self.gamma * np.amax(self.model.predict(prox_estado)[0])) #pega valor que quer chegar

            target_f = self.model.predict(estado) #pega valor que chegou
            target_f[0][acao+2*lim_cont] = target #define o valor que deseja chegar
            x[i,:] = estado
            y[i,:] = target_f
            i = i + 1
            
        self.model.fit(x, y, epochs=1, verbose=0) #treina modelo
        
    def tira_ultima_janela(self):
        if self.janela_precos.shape[0] > self.janela * self.n_variaveis:
            self.janela_precos = self.janela_precos[self.n_variaveis:] #tira os ultimos precos
    
    def remember(self, acao, reward, done):
        self.memory.append((acao, reward, self.state, self.next_state, done))
    
    def carrega_pesos(self, name):
        self.model.load_weights(name) #carrega pesos

    def salva_pesos(self, name):
        self.model.save_weights(name) #salva pesos
                
########################  DECLARA MODELO ################################
modelo = DQNAgent(n_entradas, n_saidas, epsilon, janela, n_neuronios, n_variaveis)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 320)               7680      
_________________________________________________________________
dense_2 (Dense)              (None, 256)               82176     
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_4 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_5 (Dense)              (None, 21)                5397      
Total params: 226,837
Trainable params: 226,837
Non-trainable params: 0
_________________________________________________________________


In [ ]:
########################### FUNCOES ###############################################################

def atuacao( preco, ncont, acao, custo, valor ):  #preo atual, n de contratos posicionados,
                                                #acaoo atual, custo, valor da posio
    preco_cheio = 0.
    valor_cheio = 0.
    ncont_anterior = ncont #salva posio anterior
    ncont += acao #posio atual = pos anterior + ao
    reward = 0.
    posicao = 0.
    dp = 0.

    #variaveis para tratamento do sinal de ncont, ncont_anterior e da acao
    var1 = ncont*ncont_anterior
    var2 = ncont*acao

    if acao != 0:
        #realizacao de lucros e custos
        if ((var2>0) and (var1>=0)):
            #aumento da posicao
            reward = -custo*abs(acao)    #reward = -custo*acao se houve operacao (aumento da posicao)
            #aumento da posicao = preco medio:
            valor = (ncont_anterior*valor + acao*preco)/ncont  
            
        else:
            valor_cheio = valor * pstd + pmean  #valores nao normalizados
            preco_cheio = preco * pstd + pmean
            if (var1)>= 0:
                #diminuicao da posicao
                reward = acao*(valor_cheio - preco_cheio)*10 - custo*abs(acao)  #reward se houve diminuicao da posicao
                #valor medio nao muda na diminuicao, somente se ncont==0:
                if ncont==0: valor=-10
                    
            else:
                #troca de posicao
                reward = (-ncont_anterior)*(valor_cheio - preco_cheio)*10 - custo*abs(acao)  #reward se houve troca da posicao
                #preco medio da posicao = preco da troca de posicao
                valor = preco
    
    #definindo o lucro potencial da carteira(posicao)
    if valor!=-10:
        valor_cheio = valor * pstd + pmean  #valor posicionado atual      
        dp = (preco * pstd + pmean) - valor_cheio #variacao do preco atual e do preco de compra/venda
        posicao = ncont * dp * 10 - custo*abs(ncont)           #posicao = lucro (POTENCIAL)
        
    return ncont, valor, posicao, reward

def obter_acao(ncont):
    decisao = modelo.toma_acao() #calcula a saida da rede neural
    
    acao = decisao - 2*lim_cont
    
    if abs(ncont+acao) <= lim_cont:
        return acao, 0 
    else:
        return 0, -20

def rodar_1dia(precos, custo, dia):
    global melhor_reward
    ncont = 0 #cria variavel de quantidade de contratos
    ncont_anterior = 0 #cria variavel para quantidade de contratos anterior
    valor = -10 #cria variavel para preo medio
    reward = 0. #cria variavel para recompensa
    lucro = 0.
    posicao = 0 #cria variavel de posio 
    erro = []
    modelo.limpa_memoria_dia() #limpa o vetor de memoria
    done = False
    posicao_max = 10e4
    shp = modelo.janela_precos.shape
    penal = 0
    decisao = 0
    
    for step in range( steps[dia - 1], steps[dia] ):  #roda os dados
        
        ultimos_precos = precos[ step : step + 1 ] #pega os valores de agora
        modelo.janela_precos = np.delete(np.insert(modelo.janela_precos,shp,np.concatenate(ultimos_precos.values).tolist()),[0,l])
        modelo.state = np.reshape(np.concatenate((modelo.janela_precos, [ncont/lim_cont, valor, posicao/posicao_max])), (1,n_entradas)) #adiciona na variavel de estado      
        
        acao, penal = obter_acao(ncont)        
        ncont, valor, posicao, reward = atuacao(precos['preco'][step], ncont, acao, custo, valor)
        
        #pos acao
        lucro += reward    #LUCRO ACUMULADO
        reward += penal    #penalidade se quiser ficar posicionado mais que o permitido
        
        #v=valor if valor==-10 else valor*pstd+pmean
        #p=precos['preco'][step]*pstd+pmean
        #print("acao={0}; ncont={1}; valor={2}; preco={3}; posicao={4}; lucro={5}; reward={6}".format(acao, ncont, v, p, posicao, lucro, reward))   

        if step == (steps[dia] - 1): #se ultimo step do dia
            done = True     
            prox_precos = np.zeros(n_variaveis)
            modelo.prox_janela_precos = np.delete(np.insert(modelo.prox_janela_precos,shp,prox_precos),[0,l])
        else:
            prox_precos = precos[ step + 1 : step + 2 ] #pega os proximos valores
            modelo.prox_janela_precos = np.delete(np.insert(modelo.prox_janela_precos,shp,np.concatenate(prox_precos.values).tolist()),[0,l]) 
            
        modelo.next_state = np.reshape(np.concatenate((modelo.prox_janela_precos, [ncont/lim_cont, valor, posicao/posicao_max])), (1,n_entradas)) #adiciona na variavel de estado      
        modelo.remember(acao, reward, done)  #salva step na memoria - reward = lucro instantaneo que a ação gerou
        
    #fim do dia
    #print("ncont zerados={0}; posicao={1}".format(ncont, posicao))
    if lucro > melhor_reward:
        melhor_reward = lucro
    return lucro #retorna o valor do reward

dias_pos = 0
dias_neg = 0

def rodar_dias(precos, custo):   
    global dias_pos
    global dias_neg
    sum_rewards = 0 #cria variavel de somatoria de recompensas
    
    for dia in range( 1, dias ): #loop de dias
        reward = rodar_1dia(precos, custo, dia)
        sum_rewards += reward #roda 1 dia e adiciona o total na variavel de somatoria
        
        if reward>0: 
            dias_pos += 1
        elif reward<0:
            dias_neg += 1
    return sum_rewards


In [ ]:
import time
##################################  MAIN  #########################################
if __name__ == "__main__":
    global dias_pos
    global dias_neg
    sum_rewards_total = 0
    sr=[]
    try:
        if carregar_pesos:
            modelo.carrega_pesos('./pesos.h5')
        epoca_parou = epoca_init
        for epoca in range(epoca_init, epocas+5): #rodar uma quantidade de epocas
            modelo.limpa_memoria_epoca()
            sum_rewards = rodar_dias(inputs, custo) #adiciona o resultado da epoca na somatoria
            modelo.treina_modelo() #roda o modelo com toda a memoria da epoca
            sum_rewards_total += sum_rewards
            sr.append(sum_rewards)
            print("resultado da epoca {0} = {1:0.2f} Epsilon = {2:0.3f}".format(epoca, sum_rewards, modelo.epsilon))
            print(dias_pos)
            print(dias_neg)
            epoca_parou += 1
            if ((epoca % 200) == 0):
                modelo.salva_pesos('./pesos_5.h5')
            dias_pos = 0
            dias_neg = 0            
            if modelo.epsilon <= epsilon_min:
                modelo.epsilon = epsilon_min
            else:
                modelo.epsilon -= epsilon_decay
    finally:
        print(dias_pos)
        print(dias_neg)
        modelo.salva_pesos('./pesos_5.h5')
        if carregar_epoca_epsilon:
            file = open("./epoca_epsilon.txt", "w")
            file.writelines("{0},{1}".format(epoca_parou, modelo.epsilon))
            file.close()
            print("parou na epoca {0} com epsilon {1}".format(epoca_parou, modelo.epsilon))
            
        print("Melhor resultado diario: {0:0.2f}".format(melhor_reward))
        plt.plot(range(0, len(sr)), sr, range(0, len(sr)), np.zeros(len(sr))) #plota os valores de reward por epoca

W1105 14:32:08.187637  5468 deprecation_wrapper.py:119] From D:\anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



resultado da epoca 0 = -24554.74 Epsilon = 1.000
98
121
resultado da epoca 1 = -13821.15 Epsilon = 0.999
103
116
resultado da epoca 2 = -18184.79 Epsilon = 0.998
101
118
resultado da epoca 3 = -12834.78 Epsilon = 0.997
108
111
resultado da epoca 4 = -28007.86 Epsilon = 0.996
103
116
resultado da epoca 5 = -48206.08 Epsilon = 0.995
98
121
resultado da epoca 6 = -49490.68 Epsilon = 0.994
80
139
resultado da epoca 7 = -17409.21 Epsilon = 0.993
105
114
resultado da epoca 8 = -29580.63 Epsilon = 0.992
94
125
resultado da epoca 9 = -3996.21 Epsilon = 0.991
113
106
resultado da epoca 10 = -25813.53 Epsilon = 0.990
94
125
resultado da epoca 11 = -38062.79 Epsilon = 0.989
93
126
resultado da epoca 12 = -7599.34 Epsilon = 0.988
111
108
resultado da epoca 13 = -6159.35 Epsilon = 0.987
111
108
resultado da epoca 14 = -5258.87 Epsilon = 0.986
108
111
resultado da epoca 15 = -11843.73 Epsilon = 0.985
102
117
resultado da epoca 16 = -651.49 Epsilon = 0.984
114
105
resultado da epoca 17 = -14212.62 Ep

resultado da epoca 143 = -14181.19 Epsilon = 0.858
98
121
resultado da epoca 144 = -36718.79 Epsilon = 0.857
96
123
resultado da epoca 145 = -47860.46 Epsilon = 0.856
88
131
resultado da epoca 146 = -23338.85 Epsilon = 0.855
103
116
resultado da epoca 147 = -56217.65 Epsilon = 0.854
93
126
resultado da epoca 148 = -27158.93 Epsilon = 0.853
95
124
resultado da epoca 149 = -20128.86 Epsilon = 0.852
96
123
resultado da epoca 150 = -32955.21 Epsilon = 0.851
93
126
resultado da epoca 151 = -12729.06 Epsilon = 0.851
98
121
resultado da epoca 152 = -34753.64 Epsilon = 0.850
91
128
resultado da epoca 153 = -30869.95 Epsilon = 0.849
97
122
resultado da epoca 154 = -10983.06 Epsilon = 0.848
98
121
resultado da epoca 155 = -33066.91 Epsilon = 0.847
99
120
resultado da epoca 156 = -30811.18 Epsilon = 0.846
90
129
resultado da epoca 157 = -15666.89 Epsilon = 0.845
98
121
resultado da epoca 158 = -62651.75 Epsilon = 0.844
84
135
resultado da epoca 159 = -39233.32 Epsilon = 0.843
100
119
resultado da

resultado da epoca 284 = -24509.12 Epsilon = 0.719
100
119
resultado da epoca 285 = -18780.78 Epsilon = 0.718
98
121
resultado da epoca 286 = -23946.80 Epsilon = 0.717
93
126
resultado da epoca 287 = -17376.15 Epsilon = 0.716
108
111
resultado da epoca 288 = -40980.73 Epsilon = 0.715
95
124
resultado da epoca 289 = -18736.58 Epsilon = 0.714
104
115
resultado da epoca 290 = -49006.81 Epsilon = 0.713
96
123
resultado da epoca 291 = -29619.47 Epsilon = 0.712
96
123
resultado da epoca 292 = -24037.04 Epsilon = 0.711
105
114
resultado da epoca 293 = -16810.60 Epsilon = 0.710
108
111
resultado da epoca 294 = -38450.29 Epsilon = 0.709
92
127
resultado da epoca 295 = -17796.65 Epsilon = 0.708
104
115
resultado da epoca 296 = -21527.92 Epsilon = 0.707
105
114
resultado da epoca 297 = -19987.11 Epsilon = 0.706
100
119
resultado da epoca 298 = -12088.46 Epsilon = 0.705
110
109
resultado da epoca 299 = -9138.86 Epsilon = 0.704
109
110
resultado da epoca 300 = -25136.72 Epsilon = 0.703
103
116
resu

resultado da epoca 425 = -11891.76 Epsilon = 0.579
109
110
resultado da epoca 426 = 5901.13 Epsilon = 0.578
113
106
resultado da epoca 427 = 9026.27 Epsilon = 0.577
114
105
resultado da epoca 428 = -15518.16 Epsilon = 0.576
103
116
resultado da epoca 429 = -21527.39 Epsilon = 0.575
98
121
resultado da epoca 430 = -12248.50 Epsilon = 0.574
110
109
resultado da epoca 431 = -1628.33 Epsilon = 0.573
106
113
resultado da epoca 432 = -10375.85 Epsilon = 0.572
101
118
resultado da epoca 433 = -43110.02 Epsilon = 0.571
97
122
resultado da epoca 434 = -20641.78 Epsilon = 0.570
100
119
resultado da epoca 435 = -17886.09 Epsilon = 0.569
113
106
resultado da epoca 436 = -3025.07 Epsilon = 0.568
110
109
resultado da epoca 437 = 1513.51 Epsilon = 0.567
107
112
resultado da epoca 438 = 7908.54 Epsilon = 0.566
109
110
resultado da epoca 439 = 11840.48 Epsilon = 0.565
108
111
resultado da epoca 440 = -4070.95 Epsilon = 0.564
117
102
resultado da epoca 441 = -14359.61 Epsilon = 0.563
97
122
resultado da

resultado da epoca 566 = 12777.30 Epsilon = 0.440
108
111
resultado da epoca 567 = 7657.28 Epsilon = 0.439
121
98
resultado da epoca 568 = -21257.76 Epsilon = 0.438
105
114
resultado da epoca 569 = -17592.91 Epsilon = 0.437
106
113
resultado da epoca 570 = -26816.61 Epsilon = 0.436
99
120
resultado da epoca 571 = 15356.24 Epsilon = 0.435
117
102
resultado da epoca 572 = 9635.33 Epsilon = 0.434
104
115
resultado da epoca 573 = -17411.88 Epsilon = 0.433
104
115
resultado da epoca 574 = 8930.85 Epsilon = 0.432
112
107
resultado da epoca 575 = 22235.89 Epsilon = 0.431
116
103
resultado da epoca 576 = 20214.54 Epsilon = 0.430
126
93
resultado da epoca 577 = -5850.36 Epsilon = 0.429
112
107
resultado da epoca 578 = 30645.73 Epsilon = 0.428
123
96
resultado da epoca 579 = -317.16 Epsilon = 0.427
110
109
resultado da epoca 580 = 6805.58 Epsilon = 0.426
108
111
resultado da epoca 581 = -13672.65 Epsilon = 0.425
111
108
resultado da epoca 582 = -37533.57 Epsilon = 0.424
100
119
resultado da epoc

resultado da epoca 708 = -7353.74 Epsilon = 0.299
109
110
resultado da epoca 709 = -25134.35 Epsilon = 0.298
105
114
resultado da epoca 710 = -44124.66 Epsilon = 0.297
97
122
resultado da epoca 711 = -17649.50 Epsilon = 0.296
100
119
resultado da epoca 712 = -5045.99 Epsilon = 0.295
111
108
resultado da epoca 713 = -9357.76 Epsilon = 0.294
107
112
resultado da epoca 714 = -17170.75 Epsilon = 0.293
99
120
resultado da epoca 715 = -3151.60 Epsilon = 0.292
112
107
resultado da epoca 716 = 13036.04 Epsilon = 0.291
113
106
resultado da epoca 717 = -9331.08 Epsilon = 0.290
105
114
resultado da epoca 718 = -6251.19 Epsilon = 0.289
112
107
resultado da epoca 719 = -13380.64 Epsilon = 0.288
112
107
resultado da epoca 720 = -12274.46 Epsilon = 0.287
101
118
resultado da epoca 721 = 12900.54 Epsilon = 0.286
113
106
resultado da epoca 722 = -945.55 Epsilon = 0.285
110
109
resultado da epoca 723 = -8597.86 Epsilon = 0.284
100
119
resultado da epoca 724 = 112.65 Epsilon = 0.283
108
111
resultado da 